### Relatório
Falar aqui do trabalho, introduzir a proposta

In [2]:
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt


Inicialmente, declaramos as variáveis que irão armazenar as parametrizações, além de setar alguns parametros e algumas burocracias de código. Também definimos o dominios das parametrizações.
##### Coloque a sua parametrização e o domínio nas variáveis abaixo 

In [4]:

def retornaParametrizacao (t):
    # Parametrização em X
    xParam = np.cos(t)
    # Parametrização em Y
    yParam = np.sin(t)
    return (xParam, yParam)
    

dominioMin = np.pi
dominioMax = (3*np.pi) / 2

In [12]:
v0 = 0
tTotal = 0
distTotal = 0
precisao = 0.01
delta = np.arange(dominioMin,dominioMax, precisao)
gravidade = 9.81

xMax, yMax = retornaParametrizacao(dominioMin)

-1.0 1.22464679915e-16


Depois, declaramos uma função cujo objetivo é achar os segmentos de reta que usaremos para aproximar a curva. Esses segmentos são, como mostra a imagem, a hipotenusa entre dois pontos da curva. A diferença entre esses pontos é a precisão do cálculo. Quanto menor a diferença entre eles, melhor a aproximação. A função recebe os pontos para os quais calcular o segmento de reta, obtido através da função acima. Além disso, ela retorna o angulo de inclinação das retas que irá nos ajudar a calcular o tempo.

![hipotenusa](hip.png)

In [7]:
def findHipotenusa (x0, x1, y0, y1):
    deltaX = x1 - x0
    deltaY = y1 - y0
    # Em radiano
    angulo = np.arctan(deltaY/deltaX)
    return np.sqrt((deltaX**2) + (deltaY**2)), angulo

Então, declaramos uma função que irá nos retornar o tempo de percurso de cada segmento de reta. Ela recebe como parametros o angulo de inclinação, a distancia (hipotenusa) e a velocidade inicial. E ela retorna o tempo de percurso e a velocidade final (que será utilizada como inicial para o próximo segmento na próxima iteração, e assim por diante).
Para calcular o tempo, utilizamos a equação:
![fisica](equacao.png)

Como já possuimos os parametros _ΔS_ (tamanho do segmento/hipotenusa), _v0_ (inicialmente 0, depois igual a velocidade final no segmento anterior) e _a_ (aceleração da gravidade), só precisamos isolar o t.

In [8]:
# Formula usada
# dist = v0*t + (a*(t**2))/2
def retornaTempo (teta, dist, v0):
    a = gravidade * np.cos(teta)
    
    delta = 4*(v0**2) + 8*a*dist
    t1 = (-2*v0 + np.sqrt(delta))/(2*a)
    #t2 = (-2*v0 - np.sqrt(delta))/(2*a)
    v = v0 + a*t1 
    return (t1,v)

Por fim, contruímos o loop que junta todas essa funções. Ele roda para cada intervalo de precisão definido, e tem o seguinte comportamento: 
* Primeiro, descobre as coordenadas dos dois pontos do segmento aproximado por reta, com base na precisão pré definida
* Depois, descobre o comprimento dessa reta (hipotenusa) e o seu angulo de inclinação, usando seu ponto inicial e final.
* Então, calcula o tempo necessário para percorrer tal segmento e o adiciona ao tempo total para percorrer a curva.
* Isso é repetido para cada intervalo de precisão definido, até percorrer a curva inteira.

Por fim, imprime os valores

In [13]:
for t in delta:
    x0, y0 = retornaParametrizacao(t)
    x1, y1 = retornaParametrizacao(t + precisao)
    
    if (y1 > yMax):
        print('A bolinha não consegue subir essa curva. O Y máximo é {0}'.format(yMax))
        break
    
    hip, angulo = findHipotenusa(x0,x1,y0,y1)
    distTotal += hip
    t1, v = retornaTempo(angulo, hip, v0)
    v0 = v
    tTotal += t1
    #print(t1,v)
    
print("Tempo total")
print(tTotal)
print('Distancia total')
print(distTotal)

# plt.plot(parabola)
# plt.plot(x,y)

# reta = 2*delta2
# plt.plot(reta)
#for x in delta:
# plt.show()

Tempo total
2.26049822885
Distancia total
1.57999341667
